# Customer Churn and Win-Back Targeting

Spencer Brothers

MKTG 6620-090 Fall 2025

# Business Problem

Leadership for a subscription business wants two things that can be used soon:

1. A reliable score for each active customer that estimates the chance they will cancel in the next period.

2. A simple, budget aware rule that tells the retention team whom to contact and how many to contact per 1,000 customers.

The purpose of this notebook is not only to produce a model. It is to deliver an end-to-end process that turns
raw data into a calibrated probability and then into an action a manager can follow.

# Leakage Policy

Leakage is any use of information that would not exist at the time we decide whom to contact.

Assume we score customers at the end of a billing period to plan outreach for the next 30 days.

*Only information available up to that scoring time is allowed.*

TO DO: Our report must include one paragraph that lists the leakage risks you checked and how we prevented different sources of leakage.

# Setup

In [2]:
# load necessary packages and set filepaths
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# TO DO: consolidate all necessary imports

In [1]:
# mount to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## IMPORTANT NOTE

The requirements for this assignment state that "Your notebook must run end-to-end from a clean runtime without manual fixes." However, it's impossible to access canvas files from a google colab notebook. Because of this, we will use Google Drive as an intermediary storage location. Because of this, **the data and output folders must be changed to the current user's data and output folders before running the rest of this notebook, or it will not run correctly.**

This notebook uses a common folder in Google Drive, `MKTG-6620-090-Fall-2025-Final-Project`, that contains both the data and output folders to keep everything organized.

In [3]:
# define data and output locations
data_folder = '/content/drive/MyDrive/MKTG-6620-090-Fall-2025-Final-Project/data'
output_folder = '/content/drive/MyDrive/MKTG-6620-090-Fall-2025-Final-Project/output'

# Data

## Loading `churn_train.csv`

In [5]:
churn_train = pd.read_csv(f'{data_folder}/churn_train.csv')

display(churn_train)

,ID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,yrhdgfy_4741,Male,0,No,No,35,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,49.20,1701.65,No
1,yrhdgfy_4154,Male,0,Yes,Yes,15,Yes,No,Fiber optic,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,75.10,1151.55,No
2,yrhdgfy_5863,Male,0,Yes,Yes,13,No,No phone service,DSL,Yes,...,No,Yes,No,No,Two year,No,Mailed check,40.55,590.35,No
3,yrhdgfy_4870,Female,0,Yes,No,26,Yes,No,DSL,No,...,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),73.50,1905.7,No
4,yrhdgfy_4813,Male,0,Yes,Yes,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,44.55,44.55,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,yrhdgfy_7306,Female,0,Yes,No,71,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Two year,No,Electronic check,109.25,7707.7,No
5630,yrhdgfy_7230,Male,0,No,No,2,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Bank transfer (automatic),46.05,80.35,Yes
5631,yrhdgfy_5676,Female,1,No,No,25,Yes,Yes,Fiber optic,Yes,...,No,No,Yes,Yes,Month-to-month,Yes,Mailed check,102.80,2660.2,Yes
5632,yrhdgfy_3713,Female,0,Yes,No,24,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Credit card (automatic),20.40,482.8,No


## Features

These feature descriptions are taken from the data dictionary provided with the assignment:

**ID** — Unique customer identifier.

**gender** — Male or Female.

**SeniorCitizen** — 1 if the customer is a senior citizen, 0 otherwise.

**Partner** — Customer has a partner: Yes or No.

**Dependents** — Customer has dependents: Yes or No.

**tenure** — Number of months the customer has stayed with the company.

**PhoneService** — Customer has phone service: Yes or No.

**MultipleLines** — Customer has multiple phone lines: Yes, No, or “No phone service.”

**InternetService** — Type of internet connection: DSL, Fiber optic, or No.

**OnlineSecurity** — Online security add-on: Yes, No, or “No internet service.”

**OnlineBackup** — Online backup add-on: Yes, No, or “No internet service.”

**DeviceProtection** — Device protection add-on: Yes, No, or “No internet service.”

**TechSupport** — Tech support add-on: Yes, No, or “No internet service.”

**StreamingTV** — Streaming TV add-on: Yes, No, or “No internet service.”

**StreamingMovies** — Streaming movies add-on: Yes, No, or “No internet service.”

**Contract** — Contract term: Month-to-month, One year, or Two year.

**PaperlessBilling** — Billing is paperless: Yes or No.

**PaymentMethod** — Payment method: Electronic check, Mailed check, Bank transfer
(automatic), or Credit card (automatic).

**MonthlyCharges** — Current monthly charge amount.

**TotalCharges** — Total amount charged to date (may contain blanks that must be coerced to
numeric during cleaning).

**Churn** — Target label: Yes if the customer left during the target window; No otherwise.



## Cleaning and Pre-Processing

### Converting Data Types to

### Handling nulls

In [8]:
# for each column in churn_train, count how many null values there are
churn_train.isnull().sum()

,0
ID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


It looks like there aren't any null values in the TotalCharges column that the dictionary warned about, but we can easily handle nulls in this column by replacing them with 0.

### Handling Class Imbalances

There is a class imbalance in the dataset's target (`Churn`). We can improve models' performance by handling this imbalance via weighting.


In [9]:
# show distribution of values in the Churn column
churn_train['Churn'].value_counts(normalize=True)

,proportion
Churn,
No,0.734647
Yes,0.265353


It looks like there's a moderate class imbalance, with about 27% of customers churning.

# Baseline Model (Logistic)

# Other Models and Calibration

# Decision Rule and Cost Table

# Save Figures and Files

# Holdout Scoring

# Business Recommendations

# Appendix: AI Use, Prompts, And Reflection

##